In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import helper
from tqdm import tqdm

In [2]:
train_data = helper.load_data_from_file("../data/graph_data_cleaned.json") # dictionary of SMILES and values are graphs

In [3]:
# get all the unique atom types, orbitals and their respective binding energies
atomic_energies = {}
for mol in train_data:
    graph = train_data[mol]
    atom = list(nx.get_node_attributes(graph, "atom_type").values())
    orb = list(nx.get_node_attributes(graph, "orbitals").values())
    energy = list(nx.get_node_attributes(graph, "binding_energies").values())

    for i in range(len(atom)):
        if atom[i] not in atomic_energies:
            atomic_energies[atom[i]] = {}
        for orb_type, orb_energy in zip(orb[i], energy[i]):
            if orb_type == - 1:
                continue
            if orb_type not in atomic_energies[atom[i]]:
                atomic_energies[atom[i]][orb_type] = []
            atomic_energies[atom[i]][orb_type].append(orb_energy)

In [4]:
atomic_energies['S']['1s']

[2490.1, 2483.7, 2478.5]